# Feature selection with impurity and permutation importance on the Ames Housing dataset.
In this notebook, we will compare UFI, MDI and Permutation importance on their ability to perform feature selection on the high-dimensional Ames Housing dataset, and their associated computational cost.

# Import Libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import warnings

from sklearn.compose import make_column_transformer
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder

meson-python: building scikit-learn: /home/gaeta/.sklearn-env/bin/ninja
[1/3] Generating 'sklearn/tree/_tree.cpython-312-x86_64-linux-gnu.so.p/_tree.cpp'
[2/3] Compiling C++ object sklearn/tree/_tree.cpython-312-x86_64-linux-gnu.so.p/meson-generated__tree.cpp.o
[3/3] Linking target sklearn/tree/_tree.cpython-312-x86_64-linux-gnu.so


# Data fetching

In [2]:
ames_housing = fetch_openml(data_id=42165, as_frame=True, return_X_y=False)
y = ames_housing["target"].astype(float)
ames_housing = ames_housing["data"].drop(columns="Id")
ames_housing.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


# Data preprocessing

In [3]:
numerical_features = ames_housing.select_dtypes("number").columns
categorical_features = ames_housing.columns.difference(numerical_features)

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="most_frequent")),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

preprocessor = make_column_transformer(
    (categorical_pipeline, categorical_features),
    (SimpleImputer(strategy="mean"), numerical_features),
    remainder='passthrough'
)

ames_housing_transformed = preprocessor.fit_transform(ames_housing)

feature_names = (
    categorical_features.tolist() + 
    numerical_features.tolist()
)

ames_housing_preprocessed = pd.DataFrame(
    ames_housing_transformed,
    columns=feature_names,
    index=ames_housing.index
)

# Augment the dataset with random features of varying cardinality

In [4]:
random_cat_sizes = [2, 5, 10, 20, 50, 100, 200]
n_random_num = 10
random_features = list()
n_sample = len(y)
rng = np.random.RandomState(42)

X = ames_housing_preprocessed.copy()
for cat_size in random_cat_sizes:
    X[f"random_cat_{cat_size}"] = rng.randint(0, cat_size, size=n_sample)
    random_features.append(f"random_cat_{cat_size}")
for i in range(n_random_num):
    X[f"random_num_{i}"] = rng.normal(size=n_sample)
    random_features.append(f"random_num_{i}")

X.head()

,Alley,BldgType,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,BsmtQual,CentralAir,Condition1,Condition2,...,random_num_0,random_num_1,random_num_2,random_num_3,random_num_4,random_num_5,random_num_6,random_num_7,random_num_8,random_num_9
0,0.0,0.0,3.0,3.0,2.0,5.0,2.0,1.0,2.0,2.0,...,0.725073,-1.223779,-0.240672,-1.333454,0.861844,-0.267616,-1.212832,-1.118560,0.020618,-0.285014
1,0.0,0.0,3.0,1.0,0.0,5.0,2.0,1.0,1.0,2.0,...,1.443513,-0.390764,-0.848696,-1.763622,0.737040,-1.273313,0.857255,-0.912569,0.132836,-1.998611
2,0.0,0.0,3.0,2.0,2.0,5.0,2.0,1.0,2.0,2.0,...,-0.608100,-1.174722,0.551793,1.091203,0.301727,0.384974,1.645823,-0.337125,1.258517,-0.868401
3,0.0,0.0,1.0,3.0,0.0,5.0,3.0,1.0,2.0,2.0,...,0.780452,-0.377384,-0.435901,0.662940,1.316805,-0.133807,-1.632656,0.118621,-1.470126,1.174689
4,0.0,0.0,3.0,0.0,2.0,5.0,2.0,1.0,2.0,2.0,...,-0.712072,-0.465242,0.644522,0.698413,0.197395,-0.178086,-0.020094,0.499037,-0.493514,-0.599126


# Train and optimize a `RandomForestRegressor`.

In [5]:
# Define parameter grid for min_samples_leaf
param_grid = {"min_samples_leaf": [1, 5, 20, 100], "max_features":["log2", "sqrt", 1.0], "n_estimators": [10, 50, 100, 200]}

# Initialize Random Forest
rf = RandomForestRegressor(random_state=rng, n_jobs=-1)
rf.fit(X, y)

# Set up GridSearchCV with cross-validation
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring="r2",
    return_train_score=True,
    n_jobs=-1,
    verbose=1,
)

# Fit the grid search
grid_search.fit(X, y)

# Get the best parameter and estimator
best_params = grid_search.best_params_
best_score_gs = grid_search.best_score_

print(f"Best params: {best_params}")
print(f"Best score: {best_score_gs:.3f}")

common_params = best_params
common_params["n_jobs"] = -1

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best params: {'max_features': 1.0, 'min_samples_leaf': 1, 'n_estimators': 200}
Best score: 0.850


# Feature selection procedure

In [6]:
def feature_selection_RFECV(
    method,
    common_params,
    X,
    y,
    n_fold,
    min_features_to_select,
    random_features,
    random_state,
):
    warnings.filterwarnings(
        "ignore", message=r"The number of unique classes is greater than 50% .*"
    )
    warnings.filterwarnings(
        "ignore", message=r"`sklearn.utils.parallel.delayed` should be used with*"
    )

    if method == "UFI":
        # With oob_score=True, rf will compute UFI during fit
        common_params["oob_score"] = True
        getter = "unbiased_feature_importances_"

    elif method == "MDI":
        common_params["oob_score"] = False
        getter = "feature_importances_"

    rng = np.random.RandomState(random_state)
    common_params["random_state"] = rng
    # Define a rf regressor
    rf = RandomForestRegressor(**common_params)

    start_time = time.time()

    feature_selector = RFECV(
        estimator=rf,
        min_features_to_select=min_features_to_select,
        cv=n_fold,
        scoring="r2",
        n_jobs=-1,
        importance_getter=getter,
    )

    feature_selector.fit(X, y)
    cv_results = feature_selector.cv_results_
    best_index = np.argmax(cv_results["mean_test_score"])

    run_time = time.time() - start_time
    return dict(
        init_test_score_mean=cv_results["mean_test_score"][-1],
        init_test_score_std=cv_results["std_test_score"][-1],
        best_test_score_mean=cv_results["mean_test_score"][best_index],
        best_test_score_std=cv_results["std_test_score"][best_index],
        n_random_features=sum(
            column in random_features for column in X.columns[feature_selector.support_]
        ),
        n_removed_features=len(X.columns) - feature_selector.n_features_,
        run_time=run_time,
    )


In [7]:
n_fold = 5
# Fix a common random state to have the same forests built
random_state = 42
# Keep at least 25% of features
min_features_to_select = int(X.shape[1]*0.25)

# MDI for feature selection

In [8]:
res_MDI= feature_selection_RFECV("MDI", common_params, X, y, n_fold, min_features_to_select, random_features, random_state)
print(
    f"MDI results across {n_fold} folds:\n"
    f"Total run time: {res_MDI["run_time"]:.1f}.\n"
    f"Removed {res_MDI["n_removed_features"]} features (total of {X.shape[1]}).\n"
    f"Average cross val score of the full model: {res_MDI["init_test_score_mean"]:.3f} +/- {res_MDI["init_test_score_std"]:.3f} std.\n"
    f"Average cross val score of the best model: {res_MDI["best_test_score_mean"]:.3f} +/- {res_MDI["best_test_score_std"]:.3f} std.\n"
    f"Number of random features kept by the procedure: {res_MDI["n_random_features"]:.3f} ({len(random_features)} in total)."
)

MDI results across 5 folds:
Total run time: 356.1.
Removed 40 features (total of 96).
Average cross val score of the full model: 0.851 +/- 0.024 std.
Average cross val score of the best model: 0.854 +/- 0.021 std.
Number of random features kept by the procedure: 16.000 (17 in total).


# UFI for feature selection

In [9]:
res_UFI = feature_selection_RFECV("UFI", common_params, X, y, n_fold, min_features_to_select, random_features, random_state)
print(
    f"UFI results across {n_fold} folds:\n"
    f"Total run time: {res_UFI["run_time"]:.1f}.\n"
    f"Removed {res_UFI["n_removed_features"]} features (total of {X.shape[1]}).\n"
    f"Average cross val score of the full model: {res_UFI["init_test_score_mean"]:.3f} +/- {res_UFI["init_test_score_std"]:.3f} std.\n"
    f"Average cross val score of the best model: {res_UFI["best_test_score_mean"]:.3f} +/- {res_UFI["best_test_score_std"]:.3f} std.\n"
    f"Number of random features kept by the procedure: {res_UFI["n_random_features"]:.3f} ({len(random_features)} in total)."
)

UFI results across 5 folds:
Total run time: 377.9.
Removed 72 features (total of 96).
Average cross val score of the full model: 0.851 +/- 0.024 std.
Average cross val score of the best model: 0.865 +/- 0.019 std.
Number of random features kept by the procedure: 1.000 (17 in total).


# Comparison

In [10]:
print(
    f"UFI removed {(res_UFI["n_removed_features"]/res_MDI["n_removed_features"]-1)*100:.1f}% more features than MDI,"
    f" in {(res_UFI["run_time"]/res_MDI["run_time"]-1)*100:.1f}% more time.\n"
    f"UFI kept {res_UFI["n_random_features"]/len(random_features)*100:.1f}% of the fully random features.\n"
    f"MDI kept {res_MDI["n_random_features"]/len(random_features)*100:.1f}% of the fully random features.\n"
    f"The r2 score of UFI increased by {(res_UFI["best_test_score_mean"]/res_UFI["init_test_score_mean"]-1)*100:.2f}%.\n"
    f"The r2 score of MDI increased by {(res_MDI["best_test_score_mean"]/res_MDI["init_test_score_mean"]-1)*100:.2f}%."
)

UFI removed 80.0% more features than MDI, in 6.1% more time.
UFI kept 5.9% of the fully random features.
MDI kept 94.1% of the fully random features.
The r2 score of UFI increased by 1.64%.
The r2 score of MDI increased by 0.29%.
